In [178]:
### Advanced Programming MidTerm Tutorial: PyTables 
#### Cereal Database
##### Prepared by Mehek K

### Installing PyTables:

In [3]:
pip install tables

Note: you may need to restart the kernel to use updated packages.


#### Import all necessary libraries

In [9]:
import numpy as np
import tables
import pandas as pd

- numpy for numeric ops, pandas for CSV handling, tables for HDF5
- You'll use pandas to load and prep your data before saving it in an HDF5 format.
- PyTables (via tables) will store it in a hierarchical format suitable for fast querying.

#### Load & Explore the Cereal Dataset 

In [13]:
df = pd.read_csv("/Users/mehekkumar/Downloads/cereal.csv")

In [15]:
print(df.head())

                        name mfr type  calories  protein  fat  sodium  fiber  \
0                  100% Bran   N    C        70        4    1     130   10.0   
1          100% Natural Bran   Q    C       120        3    5      15    2.0   
2                   All-Bran   K    C        70        4    1     260    9.0   
3  All-Bran with Extra Fiber   K    C        50        4    0     140   14.0   
4             Almond Delight   R    C       110        2    2     200    1.0   

   carbo  sugars  potass  vitamins  shelf  weight  cups     rating  
0    5.0       6     280        25      3     1.0  0.33  68.402973  
1    8.0       8     135         0      3     1.0  1.00  33.983679  
2    7.0       5     320        25      3     1.0  0.33  59.425505  
3    8.0       0     330        25      3     1.0  0.50  93.704912  
4   14.0       8      -1        25      3     1.0  0.75  34.384843  


•	Quickly inspect if the data loaded properly and understand what columns you are working with.

#### Define a class to describe the cereal data table:

In [72]:
class Cereal(tables.IsDescription):
    name     = tables.StringCol(50)
    mfr      = tables.StringCol(1)
    type     = tables.StringCol(1)
    calories = tables.Int32Col()
    protein  = tables.Int32Col()
    fat      = tables.Int32Col()
    sodium   = tables.Int32Col()
    fiber    = tables.Float32Col()
    carbo    = tables.Float32Col()
    sugars   = tables.Int32Col()
    potass   = tables.Int32Col()
    vitamins = tables.Int32Col()
    shelf    = tables.Int32Col()
    weight   = tables.Float32Col()
    cups     = tables.Float32Col()
    rating   = tables.Float32Col()

- Defines a schema using IsDescription, which tells PyTables how to structure the data in the HDF5 file.
- Each line specifies a column name and its type, such as Int32Col, Float32Col, or StringCol.
- PyTables needs to know the data types and structure ahead of time for efficient storage and querying.
- This maps the CSV structure to the HDF5 table format.


#### Create a new HDF5 file

In [77]:
h5file = tables.open_file("cereal_data.h5", mode="a", title="Cereal Dataset")
table = h5file.create_table("/", "cereals", Cereal, "Cereal Nutrition Data")

- Opens/creates an HDF5 file called cereal_data. h5.
- Creates a table named cereals in the root group (/) based on the Cereal schema.


#### Insert rows

In [80]:
row = table.row
for _, record in df.iterrows():
    row['name']     = record['name'].encode('utf-8')
    row['mfr']      = record['mfr'].encode('utf-8')
    row['type']     = record['type'].encode('utf-8')
    row['calories'] = int(record['calories'])
    row['protein']  = int(record['protein'])
    row['fat']      = int(record['fat'])
    row['sodium']   = int(record['sodium'])
    row['fiber']    = float(record['fiber'])
    row['carbo']    = float(record['carbo'])
    row['sugars']   = int(record['sugars'])
    row['potass']   = int(record['potass'])
    row['vitamins'] = int(record['vitamins'])
    row['shelf']    = int(record['shelf'])
    row['weight']   = float(record['weight'])
    row['cups']     = float(record['cups'])
    row['rating']   = float(record['rating'])
    row.append()

table.flush()
h5file.close()

- Iterates over each row in the DataFrame.
- Converts and assigns values to match the schema.
- .append() adds the row to the table in memory.
- .flush() writes the in-memory table to disk.
- .close() closes the HDF5 file to save changes.
- This is the data ingestion step—moving data from CSV to HDF5.
- Encoding strings is necessary because HDF5 stores strings as byte arrays.

#### Reading the Data

In [85]:
with tables.open_file("cereal_data.h5", mode="a") as file:
    table = file.root.cereals
    for row in table.iterrows():
        print(row['name'].decode(), row['calories'], "calories")

100% Bran 70 calories
100% Natural Bran 120 calories
All-Bran 70 calories
All-Bran with Extra Fiber 50 calories
Almond Delight 110 calories
Apple Cinnamon Cheerios 110 calories
Apple Jacks 110 calories
Basic 4 130 calories
Bran Chex 90 calories
Bran Flakes 90 calories
Cap'n'Crunch 120 calories
Cheerios 110 calories
Cinnamon Toast Crunch 120 calories
Clusters 110 calories
Cocoa Puffs 110 calories
Corn Chex 110 calories
Corn Flakes 100 calories
Corn Pops 110 calories
Count Chocula 110 calories
Cracklin' Oat Bran 110 calories
Cream of Wheat (Quick) 100 calories
Crispix 110 calories
Crispy Wheat & Raisins 100 calories
Double Chex 100 calories
Froot Loops 110 calories
Frosted Flakes 110 calories
Frosted Mini-Wheats 100 calories
Fruit & Fibre Dates; Walnuts; and Oats 120 calories
Fruitful Bran 120 calories
Fruity Pebbles 110 calories
Golden Crisp 100 calories
Golden Grahams 110 calories
Grape Nuts Flakes 100 calories
Grape-Nuts 110 calories
Great Grains Pecan 120 calories
Honey Graham Ohs 12

- Reopens the file and reads from the cereals table.
- Decodes string data and prints cereal names with their calorie values.
- Validates that the data was saved and is accessible


#### Querying the Data

##### Let’s find cereals with more than 10g of sugar:

In [91]:
with tables.open_file("cereal_data.h5", "a") as file:
    table = file.root.cereals
    for row in table.where("sugars > 10"):
        print(row['name'].decode(), "-", row['sugars'], "g sugar")


Apple Jacks - 14 g sugar
Cap'n'Crunch - 12 g sugar
Cocoa Puffs - 13 g sugar
Corn Pops - 12 g sugar
Count Chocula - 13 g sugar
Froot Loops - 13 g sugar
Frosted Flakes - 11 g sugar
Fruitful Bran - 12 g sugar
Fruity Pebbles - 12 g sugar
Golden Crisp - 15 g sugar
Honey Graham Ohs - 11 g sugar
Honey-comb - 11 g sugar
Lucky Charms - 12 g sugar
Muesli Raisins; Dates; & Almonds - 11 g sugar
Muesli Raisins; Peaches; & Pecans - 11 g sugar
Mueslix Crispy Blend - 13 g sugar
Post Nat. Raisin Bran - 14 g sugar
Raisin Bran - 12 g sugar
Smacks - 15 g sugar
Total Raisin Bran - 14 g sugar
Trix - 12 g sugar


Let’s find cereals with more than 5 g of fiber:¶

In [93]:
with tables.open_file("cereal_data.h5", "a") as file:
    for row in file.root.cereals.where("fiber >= 5"):
        print(row['name'].decode(), "-", row['fiber'], "g fiber")


100% Bran - 10.0 g fiber
All-Bran - 9.0 g fiber
All-Bran with Extra Fiber - 14.0 g fiber
Bran Flakes - 5.0 g fiber
Fruit & Fibre Dates; Walnuts; and Oats - 5.0 g fiber
Fruitful Bran - 5.0 g fiber
Post Nat. Raisin Bran - 6.0 g fiber
Raisin Bran - 5.0 g fiber


##### Multicondition filter

In [97]:
with tables.open_file("cereal_data.h5", mode="a") as file:
    table = file.root.cereals
    for row in table.where("(calories < 100) & (fiber > 3.0)"):
        print(row["name"].decode(), row["calories"], row["fiber"], row["rating"])


100% Bran 70 10.0 68.40296936035156
All-Bran 70 9.0 59.425506591796875
All-Bran with Extra Fiber 50 14.0 93.70491027832031
Bran Chex 90 4.0 49.12025451660156
Bran Flakes 90 5.0 53.313812255859375
Shredded Wheat 'n'Bran 90 4.0 74.47294616699219


#### -Demonstrates how to use SQL-like filtering directly within PyTables

#### Installing Tabulate

In [99]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [103]:
from tabulate import tabulate


data = []
with tables.open_file("cereal_data.h5", "a") as file:
    table = file.root.cereals
    for row in table.where("(calories < 100) & (fiber > 3.0)"):
        data.append([row["name"].decode(), row["calories"], row["fiber"], row["rating"]])


print(tabulate(data, headers=["Name", "Calories", "Fiber", "Rating"], tablefmt="pretty"))

+---------------------------+----------+-------+--------------------+
|           Name            | Calories | Fiber |       Rating       |
+---------------------------+----------+-------+--------------------+
|         100% Bran         |    70    | 10.0  | 68.40296936035156  |
|         All-Bran          |    70    |  9.0  | 59.425506591796875 |
| All-Bran with Extra Fiber |    50    | 14.0  | 93.70491027832031  |
|         Bran Chex         |    90    |  4.0  | 49.12025451660156  |
|        Bran Flakes        |    90    |  5.0  | 53.313812255859375 |
|  Shredded Wheat 'n'Bran   |    90    |  4.0  | 74.47294616699219  |
+---------------------------+----------+-------+--------------------+


#### Compressing Data HDF5 File

In [106]:
h5file.close()

In [108]:

filters = tables.Filters(complevel=5, complib="zlib")

h5file = tables.open_file("compressed_cereal_data.h5", mode="a", title="Compressed Cereal Data")
table = h5file.create_table("/", "cereals", Cereal, "Cereal Nutrition Table", filters=filters)

- Specifies compression settings: level 5 using the zlib algorithm.
- Compresses the HDF5 data to save space while keeping access efficient.


#### Indexing Columns

In [111]:
import tables

h5file = tables.open_file("cereal_data.h5", mode="a")

table = h5file.root.cereals

table.cols.sugars.create_index()

print("Index created:", table.cols.sugars.is_indexed)

h5file.close()


Index created: True


- Adds an index on the sugars column to speed up filtering.
- Just like in databases, indexing improves performance for large datasets.


#### Hierarchical Structure (Groups & Subgroups)¶
You can create folders (called groups) inside the HDF5 file. This helps organize data.

In [114]:
h5file = tables.open_file("cereal_data.h5", mode="a")

group = h5file.create_group("/", 'processed_data', 'Cleaned Cereal Data')

table = h5file.create_table(group, 'clean_table', Cereal)

row = table.row
row['name'] = 'Bran Flakes'
row['calories'] = 120
row['protein'] = 4
row['fat'] = 1
row['sugars'] = 5
row.append()
table.flush()

h5file.close()

├── raw_data/
│   └── cereal_table
├── processed_data/
│   └── clean_table
└── logs/
    └── import_log

Show how HDF5 files can be organized like folders. This is great for separating raw vs cleaned data, or versions of data.

#### Append to Group data

In [124]:
h5file = tables.open_file("cereal_data.h5", mode="a")


table = h5file.root.processed_data.clean_table 


row = table.row
row['name'] = "New Cereal"
row['calories'] = 100
row['protein'] = 3
row['fat'] = 2
row['sugars'] = 5
row.append()

table.flush()

- Adds a new clean_table inside the processed_data group.
- Adds rows to this new table, potentially with cleaned or filtered data.
- Re-opens the subgroup and appends another cereal entry.
- Shows you can keep modifying and updating data across sessions.
- 

#### Summary: 
- How to use PyTables to create, store, and query structured data using HDF5 files.
- The benefit of schema definition, efficient storage, filtering, compression, and indexing.
- How to organize your HDF5 file like a mini database.